In [ ]:
# Import OpenCV libraries and parse arguments
import cv2
import numpy as np
import argparse
 
# Construct the argument parser and parse the arguments
# This will allow us to invoke the utility from the command line using the following format
# python <code-file.py> -i <image-filename.png>
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", type=str, required=True, help="path to input image")
args = vars(ap.parse_args())

In [ ]:
# Read the image
img_rgb = cv2.imread(args["image"], 1)


In [ ]:
# Convert Image to grayscale
image_gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)


In [ ]:
# Thresholding the image
# Convert very dark areas to black completely using OTSU Binarization, and lighter areas to white 
# Black = 0
# White = 1
# Automatically calculates the lowest and highest values of an image instead of developer provided thresholds
 
ret,image_thresholded= cv2.threshold(image_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
equalize= cv2.equalizeHist(image_thresholded)


In [ ]:
# Apply Bitwise AND
# The mask contains 2 values for each pixel
# Black = 0
# White = 1
# The Bitwise AND operation retains the original pixel value when the value of the mask is 1, and converts the pixel value to black [0,0,0] where the mask is 0

image_new = cv2.bitwise_and(image_rgb, image_rgb, mask = equalize)


In [ ]:
# Thresholding the image
# Convert very dark areas to black completely using OTSU Binarization, and lighter areas to white 
# Black = 0
# White = 1
# Automatically calculates the lowest and highest values of an image instead of developer provided thresholds
 
ret,image_thresholded= cv2.threshold(image_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
equalize= cv2.equalizeHist(image_thresholded)


In [ ]:
# Remove noise - small sections that aren't blacked out yet, by using a sliding 5x5 rectangle
# Before this step, all dark areas in the original image are fully black
# Select only those pixel values that are not fully black
lower_bound=np.array([1,0,0])
upper_bound=np.array([255,255,255])
mask=cv2.inRange(image_new, lower_bound, upper_bound)

# Use a sliding 5x5 rectangle
kernel = np.ones((5,5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
image_new = cv2.bitwise_and(image_rgb, image_rgb, mask=mask)


In [ ]:
# Convert all contiguous black areas to transparent
# Convert from BGR to BGRA (to add alpha/transparency channel)
bgra = cv2.cvtColor(image_new, cv2.COLOR_BGR2BGRA)
alpha = bgra[:, :, 3]

# Use logical indexing to set alpha/transparency channel to 0 where BGR=0
alpha[np.all(bgra[:, :, 0:3] == (0, 0, 0), 2)] = 0

# Save bgra to PNG file
cv2.imwrite('bgra.png', bgra)


In [ ]:
# Analysis
# Arbitrary threshold selected by author to classify an image as clean or dirty
# [RGB = (150,150,150)]
# DirtyIndex = 0.15 or 15%
# The closer the Dirty Index is to 0, the cleaner it is. Anything higher than 0.15 is classified as dirty
LIGHT_MIN=np.array([150,150,150,255], np.uint8)
LIGHT_MAX=np.array([255,255,255,255], np.uint8)
dst=cv2.inRange(bgra, LIGHT_MIN, LIGHT_MAX)
num_light=cv2.countNonZero(dst)
print('The number of light pixels is: ' + str(num_light))
 
DARK_MIN=np.array([0,0,0,255], np.uint8)
DARK_MAX=np.array([149,149,149,255], np.uint8)
dst=cv2.inRange(bgra, DARK_MIN, DARK_MAX)
num_dark=cv2.countNonZero(dst)
print('The number of dark pixels is: ' + str(num_dark))
 
dirtyIndex = num_dark/(num_light + num_dark)
print('Dirtiness index: ' + str(round(dirtyIndex, 2))
